In [1]:
# authenticate your kaggle api key
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()

In [2]:
# Extract

api.dataset_download_files("yashgupta24/48000-movies-dataset", "./data/48000-movies-dataset/", force=True, unzip=True)

In [1]:
import pandas as pd
# movies_df = pd.read_csv("./data/48000-movies-dataset/final_data.csv", header=0)
# movies_df.head(5)
# movies_df = movies_df.drop("ReviewBody", axis=1)
# movies_df.head(5)
# movies_df.to_csv("./data/48000-movies-dataset/movies.csv", index=False)

In [5]:
# Load
from google.cloud import bigquery
import logging
from logging import INFO
import os
import sys


# Setting up logger configuration to log back information.
logging.basicConfig(format='[%(levelname)-5s][%(asctime)s][%(module)s:%(lineno)04d] : %(message)s',
                    level=INFO,
                    stream=sys.stderr)
logger: logging.Logger = logging


client = bigquery.Client()
dataset_id = f"{client.project}.48000_movies"
dataset = bigquery.Dataset(dataset_id)
dataset.location = "us-central1"
logger.info(f"Creating dataset in project {client.project}...")

dataset = client.create_dataset(dataset, exists_ok=True, timeout=30)
logger.info(f"Dataset: {dataset_id} created")

table_id = "deb-01-346205.48000_movies.movies"
file_path  = "./data/48000-movies-dataset/movies.csv"
job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.CSV,
    skip_leading_rows=1, 
    autodetect=True,
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE
)

logger.info(f"Loading file: {file_path} to table: {table_id}")
with open(file_path, "rb") as source_file:
    job = client.load_table_from_file(source_file, table_id, job_config=job_config)
    job.result()

table = client.get_table(table_id)
logger.info(f"Loaded {table.num_rows} rows and {len(table.schema)} columns to {table_id}")

[INFO ][2022-07-15 09:46:40,075][1888686876:0020] : Creating dataset in project deb-01-346205...
[INFO ][2022-07-15 09:46:40,614][1888686876:0023] : Dataset: deb-01-346205.48000_movies created
[INFO ][2022-07-15 09:46:40,616][1888686876:0034] : Loading file: ./data/48000-movies-dataset/movies.csv to table: deb-01-346205.48000_movies.movies
[INFO ][2022-07-15 09:47:06,907][1888686876:0040] : Loaded 48651 rows and 17 columns to deb-01-346205.48000_movies.movies
